**AUTHOR:** <br>
Vasilis Raptis

**DATE:** <br>
20.05.2024 

**PURPOSE:** <br>
This notebook: 
- downloads microarray data
- filters microarray data for eur, amr, afr populations -> creates lists of snps and participants
- creates microarray plink data without Y chromosome variants (causes regenine step1 to crash otherwise)
- copies the filtered lists to bucket


**NOTES:** <br>
- use *_v7.1 microarray & wgs plink data, as some known issues have been fixed. See here: https://support.researchallofus.org/hc/en-us/articles/25646444720404-Incremental-Data-Release-for-v7-Genotyping-Array-and-Short-Read-Genomic-Data 
- use plink2
- use ancestry ids created in "01_part2_pheno_by_ancestry"
- needs to be run to create the _noY plink files, copies things to be used by regenie to bucket
- **UPDATE 04.06.2024:** run on the 1:5 subsampled data
- **UPDATE 06.06.2024:** run on the 1:10 subsampled data
- **UPDATE 08.07.2024:** run on full clean data


**Setup:**

In [1]:
# libraries
library(data.table)
library(tidyverse)

## Get my bucket name
my_bucket  <- Sys.getenv("WORKSPACE_BUCKET")
## Google project name
GOOGLE_PROJECT <- Sys.getenv("GOOGLE_PROJECT")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.table::transpose()
✖ lubridate::wday()    masks data.table::wday()
✖ lubridate::week()    masks data.table::week()
✖ lubridate::yday()    masks data.table::yday()
✖ lubridate::year()    masks data.table::year()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to beco

In [2]:
# List data in my bucket folder
system(paste0("gsutil ls ", my_bucket, "/data/pheno/clean"), intern=T)
# List storage usage in workspace
system("du -h", intern=T)

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/afr_ids_clean.txt"  
[2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/afr_pheno_clean.txt"
[3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/amr_ids_clean.txt"  
[4] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/amr_pheno_clean.txt"
[5] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/eur_ids_clean.txt"  
[6] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/eur_pheno_clean.txt"
[7] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/1on10/"             
[8] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/1on5/"

[1] "126G\t./microarray/plink_v7.1" "126G\t./microarray"           
[3] "824K\t./.ipynb_checkpoints"    "126G\t."

**Inspect paths to genomic data:**

In [3]:
system(paste0("gsutil -u ", GOOGLE_PROJECT, " -m ls -lh ", "gs://fc-aou-datasets-controlled/v7/microarray/plink_v7.1/"), intern=T)

[1] "126.12 GiB  2024-04-11T20:18:48Z  gs://fc-aou-datasets-controlled/v7/microarray/plink_v7.1/arrays.bed"
[2] " 69.06 MiB  2024-04-11T20:41:50Z  gs://fc-aou-datasets-controlled/v7/microarray/plink_v7.1/arrays.bim"
[3] "  5.64 MiB  2024-04-11T20:26:14Z  gs://fc-aou-datasets-controlled/v7/microarray/plink_v7.1/arrays.fam"
[4] "TOTAL: 3 objects, 135494264236 bytes (126.19 GiB)"

**Load microarray genomic data:** <br>
**for regenie Step 1* <br>
**126.19 GiB*


In [4]:
## make folder for microarray plink data 
# system("mkdir -p ./microarray/plink_v7.1", intern=T)
## load data into workspace
# system(paste0("gsutil -u ", GOOGLE_PROJECT, " -m cp gs://fc-aou-datasets-controlled/v7/microarray/plink_v7.1/* ./microarray/plink_v7.1"), intern=T)
## check files are in workspace
# system("ls -lh ./microarray/plink_v7.1", intern=T)

**Remove Y chrom genotypes:**

In [5]:
## create array plink files without Y chromosome (regenie crashes)
# system(paste0("plink2 --bfile microarray/plink_v7.1/arrays --chr 1-22, X --make-bed --threads $(nproc) --out microarray/plink_v7.1/arrays_noY"), intern=T) 

In [6]:
## delete original array plink files to save space
# system(paste0("rm microarray/plink_v7.1/arrays.bed microarray/plink_v7.1/arrays.bim microarray/plink_v7.1/arrays.fam"), intern=T)
# system(paste0("ls ./microarray/plink_v7.1/"), intern=T)
# system(paste0("du -h "), intern=T)

In [7]:
## no. of array variants (genome-wide)
system("wc -l ./microarray/plink_v7.1/arrays_noY.bim", intern=T)

[1] "1735751 ./microarray/plink_v7.1/arrays_noY.bim"

**Load id lists:**

In [8]:
#system(paste0("gsutil cp ", my_bucket, "/data/pheno/clean/1on5/*_ids_clean_1on5.txt", " ."), intern=T)
#system(paste0("gsutil cp ", my_bucket, "/data/pheno/clean/1on10/*_ids_clean_1on10.txt", " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/data/pheno/clean/*_ids_clean.txt", " ."), intern=T)
# check in workspace
system(paste0("ls ./*_ids_clean.txt"), intern=T)

character(0)

[1] "./afr_ids_clean.txt" "./amr_ids_clean.txt" "./eur_ids_clean.txt"

**Filter microarray genomic data:** <br>
**filtering is run on the background (~30 mins each ancestry, see "02_part1_genotypes_preprocessing_20240520_162841.ipynb"). Uncomment to create .id & .snplist files again* 


In [9]:
## file names
# input files
arrays = "microarray/plink_v7.1/arrays_noY"
# output files
arrays_qc_eur = "microarray/plink_v7.1/arrays_qc_eur_clean"
arrays_qc_amr = "microarray/plink_v7.1/arrays_qc_amr_clean"
arrays_qc_afr = "microarray/plink_v7.1/arrays_qc_afr_clean"
# ancestry ids to keep
eur_ids = "eur_ids_clean.txt"
amr_ids = "amr_ids_clean.txt"
afr_ids = "afr_ids_clean.txt"

European set:

In [10]:
system(paste0("plink2 --bfile ", arrays,
             " --keep ", eur_ids,
             " --chr 1-22, X",
             " --maf 0.01 --mac 3 --geno 0.03 --hwe 1e-6 --mind 0.05",
             " --write-snplist --write-samples --no-id-header",
             " --out ", arrays_qc_eur), 
      intern=T)

[1] "PLINK v2.00a6LM 64-bit Intel (16 May 2024)     www.cog-genomics.org/plink/2.0/"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
 [2] "(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
 [3] "Logging to microarray/plink_v7.1/arrays_qc_eur_clean.log."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [4] "Options in effect:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
 [5] "  --bfile microarray/plink_v7.1/arrays_noY"                                                                                                                                                                                                                                                                    

American set:

In [11]:
# system(paste0("plink2 --bfile ", arrays,
#              " --keep ", amr_ids,
#              " --chr 1-22, X",
#              " --maf 0.01 --mac 3 --geno 0.03 --hwe 1e-6 --mind 0.05",
#              " --write-snplist --write-samples --no-id-header",
#              " --out ", arrays_qc_amr), 
#       intern=T)

African set:

In [12]:
system(paste0("plink2 --bfile ", arrays,
             " --keep ", afr_ids,
             " --chr 1-22, X",
             " --maf 0.01 --mac 3 --geno 0.03 --hwe 1e-6 --mind 0.05",
             " --write-snplist --write-samples --no-id-header",
             " --out ", arrays_qc_afr), 
      intern=T)

[1] "PLINK v2.00a6LM 64-bit Intel (16 May 2024)     www.cog-genomics.org/plink/2.0/"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
 [2] "(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
 [3] "Logging to microarray/plink_v7.1/arrays_qc_afr_clean.log."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [4] "Options in effect:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
 [5] "  --bfile microarray/plink_v7.1/arrays_noY"                                                                                                                                                                                                                                                                    

**Save lists of filtered ids and SNPs to my bucket:**

In [13]:
system(paste0("gsutil cp ./", arrays_qc_eur, ".*", " ", my_bucket, "/data/arrays/clean/all/"), intern=T)
system(paste0("gsutil cp ./", arrays_qc_afr, ".*", " ", my_bucket, "/data/arrays/clean/all/"), intern=T)
system(paste0("gsutil cp ./", arrays_qc_amr, ".*", " ", my_bucket, "/data/arrays/clean/all/"), intern=T)
# check
system(paste0("gsutil ls ", my_bucket, "/data/arrays/clean/all"), intern=T)

character(0)

character(0)

character(0)

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.id"     
[2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.log"    
[3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.snplist"
[4] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.id"     
[5] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.log"    
[6] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.snplist"
[7] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.id"     
[8] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.log"    
[9] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.snplist"

In [14]:
# cleanup in bucket
system(paste0("gsutil cp ", my_bucket, "/data/arrays/clean/all/*.log", " ", my_bucket, "/data/arrays/clean/all/logs"), intern=T)
system(paste0("gsutil rm ", my_bucket, "/data/arrays/clean/all/*.log", " "), intern=T)

system(paste0("gsutil ls ", my_bucket, "/data/arrays/clean/all/"), intern=T)
system(paste0("gsutil ls ", my_bucket, "/data/arrays/clean/all/logs"), intern=T)

character(0)

character(0)

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.id"     
[2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.snplist"
[3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.id"     
[4] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.snplist"
[5] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.id"     
[6] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.snplist"
[7] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/logs/"

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/logs/arrays_qc_afr_clean.log"
[2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/logs/arrays_qc_amr_clean.log"
[3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/logs/arrays_qc_eur_clean.log"

**Relatedness files:** <br>
see: https://support.researchallofus.org/hc/en-us/articles/4614687617556-How-the-All-of-Us-Genomic-data-are-organized#h_01GY7QXB7GGQNXMVMBZTYHMP1R

In [15]:
# system("mkdir ./relatedness", intern=T)
# system(paste0("gsutil -u ", GOOGLE_PROJECT, " -m cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/* ./relatedness"), intern=T)

In [16]:
# ## flagged related samples to remove 
# rel_to_remove <- fread("relatedness/relatedness_flagged_samples.tsv")
# nrow(rel_to_remove)
# # relatedness of pairs with a kinship score over 0.1
# rel <- fread("relatedness/relatedness.tsv")

**Flagged samples:** <br>
"549 (0.22%) samples are flagged as outliers. These samples are not necessarily problematic, because they could represent computed ancestries (or admixtures) that are not well represented in our participant pool." see: "01_Get Started with WGS Data.ipyp"

In [17]:
#system("mkdir ./qc", intern=T)
#system(paste0("gsutil -u ", GOOGLE_PROJECT, " -m cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/qc/flagged_samples.tsv ./qc/"), intern=T)
#flagged <- fread("qc/flagged_samples.tsv")
#flagged